In [11]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from pprint import pprint

In [12]:
# сделаем рандомный UserAgent
ua = UserAgent()
ua.random

# ссылка соcтоит из основного сайа и индекса html
url_1 = 'https://books.toscrape.com/'
index_1 = '/index.html'

headers = {
    "User-Agent": ua.random} 

# запускаем сессию
session = requests.Session()

In [13]:
response = session.get(url_1+index_1, headers=headers)
if response.status_code < 400:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print('Что-то пошло не так')


Если предыдущий код не выдал сообщение, значит, парсинг прошел успешно. <br>
Сначала составим список всех категорий книг

In [14]:
bc = soup.find('ul', { 'class': 'nav-list'}).findChildren('a')
books_category = {}
for el in bc[1:]:
    books_category[el.getText().strip()] = url_1 + el.get('href')

In [15]:
books_category

{'Travel': 'https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'Mystery': 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'Historical Fiction': 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'Sequential Art': 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'Classics': 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'Philosophy': 'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'Romance': 'https://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'Womens Fiction': 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'Fiction': 'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'Childrens': 'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'Religion': 'https://books.toscrape.com/catalogue/categor

Список категорий книг представляет собой словарь с парой ключ-значение: (Тема, ссылка на категорию)

Дальше попробуем спарсить все книги из каждой категории <br>
*пример такого кода можно посмотреть [тут](./parse.ipynb)*

In [16]:
all_books = []

for category, url_category in books_category.items():

    #books_in_category = []
    url_category = '/'.join(url_category.split('/')[:-1])
    index = 'index.html'

    while True:
        
        response = session.get(url_category+'/'+index, headers=headers)
        
        if response.status_code < 400:
            soup_page = BeautifulSoup(response.text, 'html.parser')

        books_list = soup_page.find_all('article', ('class', 'product_pod'))
        for i in books_list:
            book = {}
            # для записи книги берем ее название и цену
            book['name'] = i.find('h3').find('a').get('title')
            book['price'] = float(i.find('div', { 'class': 'product_price'}).find('p', { 'class': 'price_color'}).getText()[2:])
            rating = i.find('p', class_='star-rating')['class'][1]
            book['rating'] = rating
            url_book = url_category+'/'+i.find('h3').find('a').get('href')
            res = session.get(url_book)
            qua = BeautifulSoup(res.text, 'html.parser').find('th', text='Availability').find_next_sibling('td')
            book['quantity'] = int(qua.text.split()[2][1:])
            des_l = BeautifulSoup(res.text, 'html.parser').find_all('p')
            des = des_l[3].getText()
            book['description'] = des[:150]+'...' if len(des) > 150 else des
            book['category'] = category
            product_code = BeautifulSoup(res.text, 'html.parser').find('th', text='UPC').find_next_sibling('td')
            book['UPC'] = product_code.text
            
            # books_in_category.append(book)
            # all_books[category] = books_in_category
            all_books.append(book)
    
        try:
            index = soup_page.find('li', { 'class': 'next'}).find('a').get('href')
    
        except:
            break
        
    print(f'Категория {category} добавлена.')        
    #print(f'Категория {category} добавлена. В ней {len(all_books[category])} книг')



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15216\1760682076.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  qua = BeautifulSoup(res.text, 'html.parser').find('th', text='Availability').find_next_sibling('td')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15216\1760682076.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  product_code = BeautifulSoup(res.text, 'html.parser').find('th', text='UPC').find_next_sibling('td')


Категория Travel добавлена.
Категория Mystery добавлена.
Категория Historical Fiction добавлена.
Категория Sequential Art добавлена.
Категория Classics добавлена.
Категория Philosophy добавлена.
Категория Romance добавлена.
Категория Womens Fiction добавлена.
Категория Fiction добавлена.
Категория Childrens добавлена.
Категория Religion добавлена.
Категория Nonfiction добавлена.
Категория Music добавлена.
Категория Default добавлена.
Категория Science Fiction добавлена.
Категория Sports and Games добавлена.
Категория Add a comment добавлена.
Категория Fantasy добавлена.
Категория New Adult добавлена.
Категория Young Adult добавлена.
Категория Science добавлена.
Категория Poetry добавлена.
Категория Paranormal добавлена.
Категория Art добавлена.
Категория Psychology добавлена.
Категория Autobiography добавлена.
Категория Parenting добавлена.
Категория Adult Fiction добавлена.
Категория Humor добавлена.
Категория Horror добавлена.
Категория History добавлена.
Категория Food and Drink доб

Все категории книг добавлены, можно посмотреть как это выглядит

In [17]:
pprint(all_books)

[{'UPC': 'a22124811bfa8350',
  'category': 'Travel',
  'description': 'â\x80\x9cWherever you go, whatever you do, just . . . '
                 'donâ\x80\x99t do anything stupid.â\x80\x9d â\x80\x94My '
                 'MotherDuring her yearlong adventure backpacking from South '
                 'Africa...',
  'name': "It's Only the Himalayas",
  'price': 45.17,
  'quantity': 19,
  'rating': 'Two'},
 {'UPC': 'ce60436f52c5ee68',
  'category': 'Travel',
  'description': 'Acclaimed travel writer Rick Antonson sets his adventurous '
                 'compass on Mount Ararat, exploring the regionâ\x80\x99s long '
                 'history, religious mysteries, and co...',
  'name': 'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and '
          'Beyond',
  'price': 49.43,
  'quantity': 15,
  'rating': 'Four'},
 {'UPC': 'f9705c362f070608',
  'category': 'Travel',
  'description': 'To coincide with the 2016 centennial anniversary of the '
                 'National Parks Servi

Запишем эти данные в json-файл

In [21]:
import json

with open('all_book.json', 'w', encoding='UTF-8') as file:
    json.dump(all_books, file, ensure_ascii=False, separators=(',\n', ':'))

Для удобства работы представим эти данные в pd.DataFrame()

In [19]:
import pandas as pd

# создаем датафрейм
df = pd.DataFrame(all_books)
df

,name,price,rating,quantity,description,category,UPC
0,It's Only the Himalayas,45.17,Two,19,"âWherever you go, whatever you do, just . . ...",Travel,a22124811bfa8350
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Four,15,Acclaimed travel writer Rick Antonson sets his...,Travel,ce60436f52c5ee68
2,See America: A Celebration of Our National Par...,48.87,Three,14,To coincide with the 2016 centennial anniversa...,Travel,f9705c362f070608
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,Two,8,With a new foreword by Tim Ferriss â¢Thereâ...,Travel,1809259a5a5f1d8d
4,Under the Tuscan Sun,37.33,Three,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,Travel,a94350ee74deaa07
...,...,...,...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,52.65,Four,14,âDionne's expertise is evident in this finel...,Politics,2b5054a4192e9b06
996,Equal Is Unfair: America's Misguided Fight Aga...,56.86,One,12,Weâve all heard that the American Dream is v...,Politics,3968e3fbf4695d7c
997,Amid the Chaos,36.58,One,15,Some people call Eritrea the âNorth Korea of...,Cultural,bb8245f52c7cce8f
998,Dark Notes,19.19,Five,15,They call me a slut. Maybe I am.Sometimes I do...,Erotica,88c21fcd38e2486e


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1000 non-null   object 
 1   price        1000 non-null   float64
 2   rating       1000 non-null   object 
 3   quantity     1000 non-null   int64  
 4   description  1000 non-null   object 
 5   category     1000 non-null   object 
 6   UPC          1000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 54.8+ KB


Вывод: Поработали со скрепингом страницы книжного магазина. Научились работать с каждой категорией по отдельности, делать выгрузку книг по этим категориям. В итоге получили данные по 1000 книг (название книги и ее цена), которые успешно записали в json-файл и преобразовали в датафрейм для дальнейшей работы